# Ejemplo solucion problema con SQP

En este notebook trabajaremos con libreria que soluciona problemas cuadraticos con restriciones lineales (QP): https://qpsolvers.github.io/qpsolvers/quadratic-programming.html

In [ ]:
!pip install qpsolvers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 kB 19.6 MB/s eta 0:00:00


In [ ]:
import autograd.numpy as np
from autograd import grad, hessian
from qpsolvers import solve_qp

Resolveremos el problema planteado en:
https://optimization.cbe.cornell.edu/index.php?title=Sequential_quadratic_programming

Problema original
$$
\begin{array}{cl}\underset{\mathbf{x}}{\operatorname{maximize}} & \sin (x_0) \cos (x_1)+\cos (x_0)\sin (x_1) \\ \text { subject to } & 0 \leq x_0+x_1 \leq \pi \\ & x_0=x_1^{3}  
\end{array}
$$


Lo transformamos en el siguiente problema de minimizacion
$$
\begin{array}{cl}\underset{x}{\operatorname{minimize}} & -\sin (x_0) \cos (x_1)-\cos (x_0)\sin (x_1) \\ \text { subject to } & x_0+x_1 \leq \pi \\ & -x_0-x_1 \leq 0 \\ & x_0-x_1^{3} = 0
\end{array}
$$

In [ ]:
coef = 10.0 #Ayuda a evitar el malcondicionamiento del problema
def f(x): #Funcion Objetivo
  xp = x[0] - np.exp(-coef*x[0])
  return -1.0*(np.sin(xp)*np.cos(x[1])+np.cos(xp)*np.sin(x[1]))

#restriccion de igualdad
def ce(x):
  return x[0]-x[1]**3

Construir una aproximación de segundo orden en la función objetivo y de primer orden en las restricciones, en forma de Programación Cuadrática (Quadratic Programming)

$$
\begin{array}{cl}\underset{x}{\operatorname{minimize}} & \frac{1}{2} \bar{x}^{\top} P \bar{x}+q^{\top} \bar{x} \\ \text { subject to } & G \bar{x} \leq h \\ & A \bar{x}=b \\ & l_b \leq \bar{x} \leq u_b\end{array}
$$

In [ ]:
#Aproximacion de segundo orden de la funcion objetivo
P = hessian(f) #Hessiana de la funcion objetivo
q = grad(f)    #gradiente de la funcion objetivo

#Linealizacion de las restricciones
def gci(x): #gradiente de la restriccion de desigualdad
  return np.array([[1.0-coef*np.exp(-coef*x[0]),1.0], [-1.0,-1.0]])
A = grad(ce)
h = np.array([np.pi,0])

Al evaluar las restricciones de desigualdad debemos obtejer una matriz (Jacobiano)

In [ ]:
gci(np.array([1.0,1.0]))

array([[ 0.999546,  1.      ],
       [-1.      , -1.      ]])

Construimos en cada iteracion el problema cuadratico, y vamos actualizando la solucion a partir de $x - x^{(k)} = \bar{x}$, o sea $x  = \bar{x} + x^{(k)} $

In [ ]:
x = np.array([0.2,0.2**(1/3)])
lambda_threshold = 0.0001
con = 0
while True:
  con = con+1
  xk = x.copy()
  #Calculamos la Hessiana
  H = P(xk)
  #Valores propios de la Hessiana
  eig = np.linalg.eigvalsh(H)
  #Sumamos un termino en la diagonal para hacer positivos los valores propios
  if (eig >=lambda_threshold).all() == False :
    H += (lambda_threshold - min(eig))* np.identity(H.shape[0])
  #Se calcula la solucion optima del problema QP
  xbar = solve_qp(H, q(xk), gci(xk), h, A(xk), np.array([0.0]), solver="osqp")
  if np.linalg.norm(xbar)<1e-6:
    break
  x = x + xbar
print(f"Solucion QP: x = {x}, en {con} iteraciones")

Solucion QP: x = [0.66409809 0.90800398], en 6 iteraciones


Evaluamos la restriccion, sumamos el vector solucion el cual debe estar cercano al valor $\frac{\pi}{2}$

In [ ]:
ce(x), np.sum(x), np.pi/2

(-0.08452506589422637, 1.5721020724540118, 1.5707963267948966)